# Context Managers

__add explanation__

In [32]:
from time import time

class Timer:
    def __enter__(self):
        self.start = time()
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.end = time()
        print(self.end - self.start, 'seconds to run')

In [33]:
with Timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.3250179290771484 seconds to run


In [34]:
from contextlib import contextmanager

@contextmanager
def timer():
    start = time()
    yield
    end = time()
    print(end - start, 'seconds to run')

In [35]:
with timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.3074147701263428 seconds to run
